# Core Concept
* Jar包生成步骤：
    * Spring重新打包mvn生成的jar，重新组织Class文件的结构并加入Spring自己的ClassLoad加载类库，通过spring-boot-maven-plugin插件实现
    * Spring通过将依赖Jar包中的Class文件和应用自己的Class文件拉通做映射（在外部Jar中的统一的偏移量），存储在JarEntry中，然后通过Class的偏移量去查找，并完成加载
    * Launcher是真正的程序入口，它会设置合适的URLClassLoader，然后在调用应用程序的main函数
        * JarLauncher在`BOOT-INF/lib/`查找依赖包，WarLauncher在`WEB-INF/lib/`和`WEB-INF/lib-provided/`查找依赖包，两者都是在`BOOT-INF/classes`查找应用程序的Class文件
        * PropertiesLauncher在`BOOT-INF/lib/`查找`loader.properties`，加载容器所需的配置属性，也可以在`META-INF/MANIFEST.MF`通过`Loader-Home`指定它的位置
            * `loader.properties`中通过`loader.home`或`loader.config.location`指定应用程序的配置文件
        * Launcher是在`META-INF/MANIFEST.MF`的`Main-Class`属性中指定，`Start-Class`属性指定应用程序的主函数类
* spring-boot-starter-parent依赖管理：
    * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-dependencies/pom.xml
* spring-boot-gradle-plugin基础插件，可以加强其他插件的功能，例如java插件，war插件，io.spring.dependency-management插件（依赖管理）
    * https://github.com/spring-projects/spring-boot/tree/master/spring-boot-project/spring-boot-starters
* spring-boot-devtools：开发帮组工具插件，只要classpath发生变化就会重启服务（通过将类分别加载到base classloader和restart classload实现）
    * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-devtools/src/main/java/org/springframework/boot/devtools/env/DevToolsPropertyDefaultsPostProcessor.java
* 特殊注解符：
    * `@SpringBootApplication`：开启自动配置，相当于开启了`@EnableAutoConfiguration，@ComponentScan，@Configuration`
    * `@Configuration`：基于java的配置，配置额外的JavaBean
    * `@ComponentScan`：指定JavaBean的扫描路径，自动加载并完成注入`@Component, @Service, @Repository, @Controller`标注的JavaBean
    * `@Import`：导入其他JavaBean的配置
    * `@ImportResource`：导入其他XML文件配置
    * `@Enable*`：开启某些功能注解格式
        * `@EnableAutoConfiguration(exclude={DataSourceAutoConfiguration.class})，spring.autoconfigure.exclude`：开启自动配置，根据依赖的Jar自动注入JavaBean，使用exclude过滤某些类，不开启自动配置
* 启动命令：
    * Run：`java -jar xxx.jar，mvn spring-boot:run，gradle bootRun`
    * Debug：`java -Xdebug -Xrunjdwp:server=y,transport=dt_socket,address=8000,suspend=n -jar xxx.jar`
* 核心类
    * SpringApplication：
        * SpringApplicationBuilder：SpringApplication的构造器
        * `META-INF/`：该文件夹存储的文件配置是ApplicationContext加载相关Bean时使用的配置
        * ApplicationStartingEvent：容器启动，注册listener和初始化构造器
        * ApplicationEnvironmentPreparedEvent：为ApplicationContext准备Environment属性
        * ApplicationPreparedEvent：加载完Bean的定义，但是Bean还没有刷新
        * ApplicationStartedEvent：ApplicationContex刷新结束，所有Bean加载完成
        * ApplicationReadyEvent：应用程序启动完成
        * ApplicationFailedEvent：初始化过程中出现任何错误将产生该事件
        * ApplicationContext类型推断：
            * Spring MVC：AnnotationConfigServletWebServerApplicationContext
            * Spring WebFlux：AnnotationConfigReactiveWebServerApplicationContext
            * 其他的：AnnotationConfigApplicationContext
            * setWebApplicationType(WebApplicationType)：手动指定ApplicationContext类型
    * ApplicationRunner和CommandLineRunner接口：在`SpringApplication.run(…)`执行结束之前执行，做些Hack事情
    * Environment管理系统配置文件属性集，以下是可配的属性参数配置文件以及顺序：列表前面的会覆盖列表后面的属性
        * 当使用了devtool插件时，首先搜索~/.spring-boot-devtools.properties
        * @TestPropertySource指定测试环境使用的配置属性
        * @SpringBootTest#properties指定测试环境使用的配置属性
        * 命令行参数，例如`java -jar myapp.jar --xxx=value`
        * SPRING_APPLICATION_JSON指定的json格式的属性值，通过命令行传入，或者spring.application.json指定
            * `SPRING_APPLICATION_JSON='{"acme":{"name":"test"}}' java -jar myapp.jar`
            * `java -Dspring.application.json='{"name":"test"}' -jar myapp.jar`：JVM系统参数
            * `java -jar myapp.jar --spring.application.json='{"name":"test"}'`：命令行参数
        * ServletConfig初始化参数，在web.xml中指定
        * ServletContext初始化参数，在web.xml中指定
        * 通过JNDI指定配置属性，例如：java:comp/env
        * 通过Java系统参数指定，主要是JVM参数，例如`java -Dxxx=value -jar myapp.jar`
        * 通过操作系统的环境变量指定，例如：export var=value
        * 通过RandomValuePropertySource产生数据属性配置
        * `application-{profile}.properties`或`application-{profile}.yml`相关：
            * spring.profiles.active属性指定具体生效的profile
            * spring.profiles.include属性可以让生效的profile中包含更多的profile
            * Jar包外的`application-{profile}.properties`或`application-{profile}.yml`配置文件
            * Jar包内的`application-{profile}.properties`或`application-{profile}.yml`配置文件
            * Jar包外的`application.properties`或`application.yml`配置文件
            * Jar包内的`application.properties`或`application.yml`配置文件
            * 默认查找路径：
                * SpringApplication主类所在目录下的`/config`目录
                * SpringApplication主类所在目录
                * classpath下的`/config`目录，一般指resource文件夹中
                * classpath根目录
            * spring.config.name指定配置文件的默认文件名
            * spring.config.location指定配置文件的默认搜索路径
                * 如果指定了该属性，配置文件的搜索顺序是该属性指定的逆序搜索
                * 如果指定了该属性，spring默认的搜索路径就被覆盖了，也就是不会再去原来的默认搜索路径查找
            * spring.config.additional-location指定额外的配置文件搜索路径，不会覆盖默认的搜索路径，也是逆序的
            * YamlPropertiesFactoryBean将yml解析成properties，YamlMapFactoryBean将yml解析成map，YamlPropertySourceLoader将yml属性暴露给Environment
        * @Configuration标注的配置类通过添加@PropertySource添加额外的配置信息
            * @PropertySource不能指定yml文件只能指定properties作为配置文件
        * 默认配色文件，通过SpringApplication.setDefaultProperties指定
    * @Value和@ConfigurationProperties注入配置属性，将Environment中的值注入到应用中
        * @Value直接标注在需要注入的属性上，支持SpEL表达式
        * @ConfigurationProperties的匹配规则不是非常严格，会自动转换驼峰命名和小写加-符号的命名风格
        * @ConfigurationProperties标注在属性类上，并且声明该类为Bean就可以将配置文件中的整个层级属性注入到该Bean中
        * @ConfigurationProperties标注在方法上，并且声明该方法为Bean就可以将配置文件中的整个层级属性注入到该方法返回的Bean中
    * Logging：默认集成Logback日志框架
        * logging.file：指定输出日志文件名
        * logging.path：指定输出日志文件路径
        * logging.file.max-size：指定日志文件大小，达到最大值后，会被打包，然后从头开始一个新的
        * logging.file.max-history：如果不指定该值，日志文件会无限循环创建过程
        * logging.level.xxx：指定xxx的日志级别
        * `logback-spring.xml`中的springProperty标签，可以让日志文件引用`application.properties`中的配置参数
* springboot相关的所有配置，自动加载，测试注解解释信息：https://docs.spring.io/spring-boot/docs/2.1.0.M1/reference/htmlsingle/#appendix
* springboot工具手册：https://docs.spring.io/spring-boot/docs/2.1.0.M1/reference/htmlsingle/#howto

****

# AutoConfig & Starter
* AutoConfig通过标准的`@Configuration`指定配置类，通过`@Conditional, @ConditionalOnClass, @ConditionalOnMissingBean`指定AutoConfig开启的条件，通过`META-INF/spring.factories`表明需要SpringBoot进行自动配置加载的`@Configuration`配置类
    * `@ConditionalOnClass, @ConditionalOnMissingBean`：分别代表某些特定的类存在或者不存在时，才触发AutoConfig，支持类级别和方法级别的注解，推荐使用类级别，原因是加载顺序问题
    * `@ConditionalOnProperty`：定义当某些属性存在时，触发AutoConfig
    * `@ConditionalOnResource`：定义当某些资源存在时，触发AutoConfig
    * `@ConditionalOnWebApplication, @ConditionalOnNotWebApplication`：定义是Web应用或者不是Web应用时，触发AutoConfig
    * `@ConditionalOnExpression`：定义当满足SpEL表达式时，触发AutoConfig
    * `ApplicationContextRunner, WebApplicationContextRunner, ReactiveWebApplicationContextRunner`：测试自定义的AutoConfig类是否设置成功
* AutoConfig支持的依赖：
    * https://github.com/spring-projects/spring-boot/tree/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure
    * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/resources/META-INF/spring.factories
* 自定义Starter步骤：
    * 创建AutoConfig模块，提供依赖的自动配置信息，以及所有使用该Starter所需的依赖
    * Starter模块只是简单的将AutoConfig模块作为依赖加入

****

# SpringBoot结合Spring MVC：
* 自动完成的配置内容
    * 使用HttpMessageConverter解析requests和responses，自动将对象转换成JSON对象
    * @JsonComponent自动序列化和反序列化JSON对象
    * MessageCodesResolver产生错误信息已经对应的错误code
    * 静态文件默认文件夹：classpath下的`resources/static或resources/public或/resources或/META-INF/resources`，可以通过`spring.mvc.static-path-pattern`更换
    * 默认关闭后缀匹配，可通过`spring.mvc.contentnegotiation.favor-path-extension`开启
    * `WebBindingInitializer`：初始化`WebDataBinder`处理参数绑定
    * 模板引擎解析默认路径：`src/main/resources/templates`
    * ErrorController负责处理错误信息请求，但请求是浏览器则会返回静态的html页面，其他的请求则会返回json对象。扩展该类可以自定义错误信息或者通过@ControllerAdvice的方式重新定义错误输出
    * 错误页面默认在classpath的`/error`目录中，例如`resources/static/error或resources/public/error`，必须以状态码命名的文件，会自动匹配对应的状态码文件返回错误信息，通过扩展ErrorViewResolver接口可以自定义错误页面
    
****

# SpringBoot结合Spring WebFlux：
* 自动完成的配置内容：
    * 使用HttpMessageReader和HttpMessageWriter解析requests和responses，通过CodecConfigurer自定义配置
    * 静态文件的解析路径与Spring MVC一致，可以通过ReServletListenerRegistrationBeansourceWebHandler配置，`spring.webflux.static-path-pattern`可以指定默认路径
    * 模板引擎默认解析路径与Spring MVC一致，在`src/main/resources/templates`
    * WebExceptionHandlerc处理异常事件，继承AbstractErrorWebExceptionHandler或DefaultErrorWebExceptionHandler可以更改默认的异常处理类
    * 错误页面与Spring MVC一致，必须以状态码命名的文件
    
****

# SpringBoot嵌入式容器
* 支持Tomcat，Jetty，Undertow
* 如果应用只有唯一的Servlet则会映射到`/`路径，否则会以Servlet名字作为路径的前缀；如果只有唯一的Filter，则会映射到`/*`
* ServletRegistrationBean，FilterRegistrationBean，ServletListenerRegistrationBean注册Servlets，Filters和Listeners到容器
* 嵌入式容器初始化顺序：
    * `org.springframework.boot.web.servlet.ServletContextInitializer`的`onStartup`方法初始化ApplicationContext
    * `@ServletComponentScan`可以自动扫描并注入`@WebServlet, @WebFilter, @WebListener`标注的Bean
    * 嵌入式容器的上下文是`ServletWebServerApplicationContext`类，根据classpath选择的嵌入式容器依赖，该类会自动创建`TomcatServletWebServerFactory, JettyServletWebServerFactory, UndertowServletWebServerFactory`三个工厂中的一个
    * 嵌入式容器的配置属性集合（服务器端口、地址等配置信息）：https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/web/ServerProperties.java
        * 可以在application.properties中修改
        * 可以通过实现`WebServerFactoryCustomizer<ConfigurableServletWebServerFactory>`接口修改
        * 可以直接创建`TomcatServletWebServerFactory, JettyServletWebServerFactory, UndertowServletWebServerFactory`类型的Bean修改
        
****

# SpringBoot处理数据
* SQL型数据库：自动配置，可以直接@Autowire
    * 数据库连接池：HikariCP，Tomcat数据库连接池，Commons DBCP2；spring-boot-starter-jdbc或spring-boot-starter-data-jpa默认使用HikariCP，可以通过`spring.datasource.type`修改数据库连接池类型
    * `DataSource`：该接口负责数据库的连接，属性集合：https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/jdbc/DataSourceProperties.java
        * 一般来说`spring.datasource.driver-class-name`属性可以从`spring.datasource.url`中反推出来
        * `spring.datasource.hikari.*, spring.datasource.tomcat.*, and spring.datasource.dbcp2.*`：为不同的数据库连接池指定特定的属性值
    * 嵌入式内存数据库：H2，HSQL，Derby；如果不指定`spring.datasource.url`则会使用该类型的数据库
    * `JdbcTemplate`：默认已经自动完成了注入，可以通过`spring.jdbc.template.*`修改其属性
    * `JPA`：`spring-boot-starter-data-jpa`依赖指定了三种JPA的实现方式，Hibernate，Spring Data JPA，Spring ORMs
        * `Spring Data JPA`：定义了一系列接口，可以通过方法名生成对应的SQL语句，`spring.jpa.*`配置其相关属性
        * `Hibernate`：`spring.jpa.properties.*`配置Hibernate的JPA属性
* NOSQL型数据库：自动配置，可以直接@Autowire
    * 支持给`Redis, MongoDB, Neo4j, Elasticsearch, Solr, Cassandra, Couchbase, LDAP`提供自动配置功能
    * Redis：使用`spring-boot-starter-data-redis`或`spring-boot-starter-data-redis-reactive`依赖，默认使用Lettuce客户端连接，但是也支持Jedis客户端连接
        * `LettuceClientConfigurationBuilderCustomizer`配置Lettuce客户端，`JedisClientConfigurationBuilderCustomizer`配置Jedis客户端
        * 操作类：`RedisConnectionFactory, StringRedisTemplate, RedisTemplate`
    * MongoDB：使用`spring-boot-starter-data-mongodb`或`spring-boot-starter-data-mongodb-reactive`依赖
        * 操作类：`MongoTemplate`，或者通过Spring Data JPA操作
    * Neo4j：使用`spring-boot-starter-data-neo4j`依赖
        * 操作类：`Neo4jTemplate`，或者通过Spring Data JPA操作，`@NodeEntity`
    * Solr：使用`spring-boot-starter-data-solr`依赖
        * 操作类：`SolrClient`，或者通过Spring Data JPA操作，`@SolrDocument`
    * Elasticsearch：使用`spring-boot-starter-data-elasticsearch`依赖
        * 操作类：`RestClient, RestHighLevelClient, JestClient, ElasticsearchTemplate, TransportClient`，或者通过Spring Data JPA操作，`@Document`
    * Cassandra：使用`spring-boot-starter-data-cassandra`依赖
        * 操作类：`CassandraTemplate`
    * Couchbase：使用`spring-boot-starter-data-couchbase`或`spring-boot-starter-data-couchbase-reactive`依赖
        * 操作类：`CouchbaseTemplate`
    * LDAP：使用`spring-boot-starter-data-ldap`依赖
        * 操作类：`LdapTemplate`
* 缓存：自动配置，可以直接@Autowire
    * 支持的缓存种类：`Generic, JCache, EhCache 2.x, Hazelcast, Infinispan, Couchbase, Redis, Caffeine, Simple`，通过`spring.cache.type`指定类型，缓存使用`spring-boot-starter-cache`依赖
    * 核心接口：`CacheManager`和`Cache`，每个不同的缓存提供商都有自己的`CacheManager`管理所有的缓存，例如`JCacheCacheManager, EhCacheCacheManager, CouchbaseCacheManager, RedisCacheManager`等等，springboot会根据配置文件所提供的缓存提供商信息自动注入对应的`CacheManager`
* 消息系统：自动配置，可以直接@Autowire
    * 支持的消息系统种类：`JMS, ActiveMQ, Artemis, RabbitMQ, Kafka`
    * 操作类：`JmsTemplate, AmqpTemplate, KafkaTemplate`
    * 配置属性集合：
        * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/jms/activemq/ActiveMQProperties.java
        * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/jms/artemis/ArtemisProperties.java
        * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/amqp/RabbitProperties.java
        * https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/kafka/KafkaProperties.java
        
**** 

# SpringBoot处理其他信息
* REST Client：不是自动配置的，需要自定义配置
    * RestTemplate：通过RestTemplateBuilder生成，与全局Server使用相同的HttpMessageConverters
    * WebClient：通过WebClient.Builder生成，与全局Server使用相同的HttpMessageReaders和HttpMessageWriters
* Email Client：自动配置，可以直接@Autowire
    * JavaMailSender，依赖`spring-boot-starter-mail`
    * 配置属性集合：https://github.com/spring-projects/spring-boot/blob/v2.1.0.M1/spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/mail/MailProperties.java
* 分布式事务，JTA，利用XA协议完成两阶段提交
    * 事务管理器：分为`Atomikos, Bitronix, Narayana`几类，由`JtaTransactionManager`管理
    * `Atomikos`：依赖`spring-boot-starter-jta-atomikos`，自动配置，可以直接@Autowire
    * `Bitronix`：依赖`spring-boot-starter-jta-bitronix`，自动配置，可以直接@Autowire
    * `Narayana`：依赖`spring-boot-starter-jta-narayana`，自动配置，可以直接@Autowire
* Spring Integration：依赖`spring-boot-starter-integration`
* Quartz Scheduler：定时器触发执行任务
* Spring Session：持久化Session，通过`JDBC, Redis, Hazelcast, MongoDB`这几种方式实现
* Spring Testing：依赖`spring-boot-starter-test`，该依赖集成了`JUnit, Spring Test & Spring Boot Test, AssertJ, Hamcrest,  Mockito, JSONassert, JsonPath`
    * 可测试类别以及范围：`@SpringBootTest, @AutoConfigureXXX, @JsonTest, @WebMvcTest, @WebFluxTest, @DataJpaTest, @JdbcTest, @JooqTest, @DataMongoTest, @DataNeo4jTest, @DataRedisTest, @DataLdapTest, @RestClientTest`
* WebSockets：依赖`spring-boot-starter-websocket`，自动配置，可以直接@Autowire
* WebServices：依赖`spring-boot-starter-webservices`，自动配置，可以直接@Autowire

****

# SpringBoot监控应用
* 依赖`spring-boot-starter-actuator`，开启应用监控，SpringBoot会创建额外的入口，提供对应用的监控
    * 接口详情：https://docs.spring.io/spring-boot/docs/2.1.0.M1/reference/htmlsingle/#production-ready
    * 访问接口需要添加`/actuator`前缀，例如`/actuator/health`
    * 通过`management.endpoint.<id>.enable`开启某些接口，`management.endpoints.enabled-by-default`设置默认全部开启或关闭
    * `management.endpoints.jmx.exposure.exclude, management.endpoints.jmx.exposure.include, management.endpoints.web.exposure.exclude, management.endpoints.web.exposure.include`：选择暴露哪些接口
    * `management.endpoints.web.path-mapping.<id>`：更改接口路径
    * `management.endpoints.web.base-path`：更改接口的基础路径
    * `management.endpoints.web.cors.allowed-origins, management.endpoints.web.cors.allowed-methods`：设置跨域请求
    * 自定义监控接口：`@Endpoint, @ReadOperation, @WriteOperation, @DeleteOperation, @JmxEndpoint, @WebEndpoint, @EndpointWebExtension, @EndpointJmxExtension`
* Health check：
    * endpoint：`/health`
    * `management.endpoint.health.show-details`：配置是否显示应用健康的详细信息，可选值`never, when-authorized, always`
    * `management.endpoint.health.roles`：配置可以查看应用健康详细信息的角色
    * 健康详细信息需要通过`HealthIndicatorRegistry`注册，具体的内容通过`HealthIndicator`加载，可以自己实现加载自定义的信息；默认加载的`HealthIndicator`包括：`CassandraHealthIndicator, DiskSpaceHealthIndicator, DataSourceHealthIndicator, RedisHealthIndicator`等等，可以通过`management.health.defaults.enabled`关闭这些默认的`HealthIndicator`
    * WebFlux使用`ReactiveHealthIndicatorRegistry, ReactiveHealthIndicator`完成类似的功能
* 应用状态信息：
    * endpoint：`/info`
    * 通过`InfoContributor`记录应用状态信息，可以自己实现加载自定义的信息；默认会加载`EnvironmentInfoContributor, GitInfoContributor, BuildInfoContributor`三种应用状态信息
    * 通过在`application.properties`文件中配置`info.*`开头的信息可以通过`EnvironmentInfoContributor`完成加载
    * `git.properties`文件中的信息通过`GitInfoContributor`完成加载，如果在`application.properties`文件配置了`management.info.git.mode=full`则会暴露`git.properties`文件中的所有属性
    * `META-INF/build-info.properties`文件中的信息通过`BuildInfoContributor`完成加载
* SSL配置
    * server.port=8443
    * server.ssl.enabled=true
    * server.ssl.key-store=classpath:main.jks
    * server.ssl.key-password=secret
    * management.server.port=8080
    * management.server.ssl.enabled=true
    * management.server.ssl.key-store=classpath:management.jks
    * management.server.ssl.key-password=secret
* 集成其他监控框架：`Atlas, Datadog, Ganglia, Graphite, Influx, JMX, New Relic, Prometheus, SignalFx, Simple, StatsD, Wavefront`
* 部署方式：`Cloud(Cloud Foundry), Heroku, OpenShift, AWS, Google Cloud`